## This experiment was performed using LightFM which a very popular recommender module and it has support to take in different data modalities such as text, image, graphical, etc. Please check out their official documentation in the link mentioned below:  

The objective of this experiment is to find the best parameters which give the best precision@12 for the problem at hand.

Please refer to notebooks where you will be able to visualize different experimentations based on light FM:  
1. Light FM with only customer article interactions:  
Link: https://www.kaggle.com/rickykonwar/h-m-lightfm-nofeatures  

2. Light FM with customer article interaction + 1 article feature (product group name)  
Link: https://www.kaggle.com/rickykonwar/h-m-lightfm-1articlefeature  

3. Light FM with customer article interaction + 1 article feature (product group name) + article description embeddings  
Link: https://www.kaggle.com/rickykonwar/h-m-lightfm-2articlefeatures  

Link to LightFM documentation
making.lyst.com/lightfm/docs/home.html  

It incorporates Hyper Parameter tuning for the problem statement

Hope you like this notebook, please feel free to vote for this notebook

## Importing Required Libraries

In [1]:
# Importing Libraries
import sys, os
import re
import tqdm
import time
import pickle
import random
import itertools

import pandas as pd
import numpy as np
import scipy.sparse as sparse
%matplotlib inline
import matplotlib.pyplot as plt

# lightfm 
from lightfm import LightFM # model
from lightfm.evaluation import precision_at_k
from lightfm.cross_validation import random_train_test_split

# multiprocessing for inferencing
from multiprocessing import Pool

In [2]:
os.environ["openblas_set_num_threads"] = "1"
data_path = r'../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
customer_data_path = r'../input/h-and-m-personalized-fashion-recommendations/customers.csv'
article_data_path = r'../input/h-and-m-personalized-fashion-recommendations/articles.csv'
submission_data_path = r'../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv'

In [3]:
# Data Extraction
def create_data(datapath, data_type=None):
    if data_type is None:
        df = pd.read_csv(datapath)
    elif data_type == 'transaction':
        df = pd.read_csv(datapath, dtype={'article_id': str}, parse_dates=['t_dat'])
    elif data_type == 'article':
        df = pd.read_csv(datapath, dtype={'article_id': str})
    return df

In [4]:
#%%time

# Load all sales data (for 3 years starting from 2018 to 2020)
# ALso, article_id is treated as a string column otherwise it 
# would drop the leading zeros while reading the specific column values
transactions_data=create_data(data_path, data_type='transaction')
print(transactions_data.shape)

# # Unique Attributes
print(str(len(transactions_data['t_dat'].drop_duplicates())) + "-total No of unique transactions dates in data sheet")
print(str(len(transactions_data['customer_id'].drop_duplicates())) + "-total No of unique customers ids in data sheet")
print(str(len(transactions_data['article_id'].drop_duplicates())) + "-total No of unique article ids courses names in data sheet")
print(str(len(transactions_data['sales_channel_id'].drop_duplicates())) + "-total No of unique sales channels in data sheet")

(31788324, 5)
734-total No of unique transactions dates in data sheet
1362281-total No of unique customers ids in data sheet
104547-total No of unique article ids courses names in data sheet
2-total No of unique sales channels in data sheet


In [5]:
transactions_data.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [6]:
transactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.2+ GB


In [7]:
%%time

# Load all Customers
customer_data=create_data(customer_data_path)
print(customer_data.shape)

print(str(len(customer_data['customer_id'].drop_duplicates())) + "-total No of unique customers ids in customer data sheet")

(1371980, 7)
1371980-total No of unique customers ids in customer data sheet
CPU times: user 3.66 s, sys: 492 ms, total: 4.15 s
Wall time: 5.72 s


In [8]:
customer_data.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [9]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [10]:
%%time

# Load all Customers
article_data=create_data(article_data_path, data_type='article')
print(article_data.shape)

print(str(len(article_data['article_id'].drop_duplicates())) + "-total No of unique article ids in article data sheet")

(105542, 25)
105542-total No of unique article ids in article data sheet
CPU times: user 750 ms, sys: 62 ms, total: 812 ms
Wall time: 1.11 s


In [11]:
article_data.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [12]:
article_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  object
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

## Capturing Seasonal Effect by Limiting the transaction date
Based on notebook with link: https://www.kaggle.com/tomooinubushi/folk-of-time-is-our-best-friend/notebook

In [13]:
transactions_data = transactions_data[transactions_data['t_dat'] > '2020-08-21']
transactions_data.shape

(1190911, 5)

## Aggregating Customers and Articles irrespective of transaction dates

In [14]:
transactions_data = transactions_data.groupby(['customer_id','article_id']).agg({'price':'sum','t_dat':'count'}).reset_index()
transactions_data = transactions_data[['customer_id','article_id','price','t_dat']]

## Generating user and article index mapping dictionaries

In [15]:
def get_customers_list():
    # Creating a list of users
    return np.sort(customer_data['customer_id'].unique())

def get_articles_list():
    # Creating a list of courses 
    item_list = article_data['article_id'].unique()
    return item_list

def id_mappings(customers_list, articles_list):
    """
    
    Create id mappings to convert user_id, item_id, and feature_id
    
    """
    customer_to_index_mapping = {}
    index_to_customer_mapping = {}
    for customer_index, customer_id in enumerate(customers_list):
        customer_to_index_mapping[customer_id] = customer_index
        index_to_customer_mapping[customer_index] = customer_id
        
    article_to_index_mapping = {}
    index_to_article_mapping = {}
    for article_index, article_id in enumerate(articles_list):
        article_to_index_mapping[article_id] = article_index
        index_to_article_mapping[article_index] = article_id
        
    return customer_to_index_mapping, index_to_customer_mapping, \
           article_to_index_mapping, index_to_article_mapping

In [16]:
customers = get_customers_list()
articles = get_articles_list()

In [17]:
customers

array(['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       '0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa',
       '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       ...,
       'ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264',
       'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
       'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
      dtype=object)

In [18]:
articles

array(['0108775015', '0108775044', '0108775051', ..., '0956217002',
       '0957375001', '0959461001'], dtype=object)

In [19]:
# Generate mapping, LightFM library can't read other than (integer) index
customer_to_index_mapping, index_to_customer_mapping, \
article_to_index_mapping, index_to_article_mapping = id_mappings(customers, articles)

## Generate Customer Article Interaction Matrix

In [20]:
def get_customer_article_interaction(customer_article_amt_df, agg_col_name='price'):
    #start indexing
    customer_article_amt_df["customer_id"] = customer_article_amt_df["customer_id"]
    customer_article_amt_df["article_id"] = customer_article_amt_df["article_id"]
    customer_article_amt_df[agg_col_name] = customer_article_amt_df[agg_col_name]

    # Preprocessing dataframe created
    customer_article_amt_df = customer_article_amt_df.rename(columns = {'price':'total_amount_spent', 't_dat': 'total_no_of_transactions'})

    # Replace Amount Column with category codes 
    if agg_col_name.__eq__('price'):
        customer_article_amt_df['total_amount_spent'] = customer_article_amt_df['total_amount_spent'].astype('category')
        customer_article_amt_df['total_amount_spent'] = customer_article_amt_df['total_amount_spent'].cat.codes
    elif agg_col_name.__eq__('t_dat'):
        customer_article_amt_df['total_no_of_transactions'] = customer_article_amt_df['total_no_of_transactions'].astype('category')
        customer_article_amt_df['total_no_of_transactions'] = customer_article_amt_df['total_no_of_transactions'].cat.codes

    return customer_article_amt_df

def get_interaction_matrix(df, df_column_as_row, df_column_as_col, 
                        df_column_as_value, row_indexing_map, col_indexing_map):
    
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    value = df[df_column_as_value].values
    
    return sparse.coo_matrix((value, (row, col)), shape = (len(row_indexing_map), len(col_indexing_map)))


### Customer Article Interaction based on Amount Spent

In [21]:
# Create customer and article interaction dataframe based on total amount spent
customer_to_article_amt = get_customer_article_interaction(customer_article_amt_df = transactions_data[['customer_id','article_id','price']])
print(customer_to_article_amt.shape)                                                  

(1051730, 3)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [22]:
customer_to_article_amt.head()

,customer_id,article_id,total_amount_spent
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043,4205
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007,4791
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0448509014,3443
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0719530003,2792
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0516859008,939


In [23]:
# Generate customer_article_interaction_matrix for train data
customer_to_article_interaction_amt = get_interaction_matrix(customer_to_article_amt, "customer_id", "article_id", "total_amount_spent", \
                                                            customer_to_index_mapping, article_to_index_mapping)

In [24]:
customer_to_article_interaction_amt

<1371980x105542 sparse matrix of type '<class 'numpy.int16'>'
	with 1051730 stored elements in COOrdinate format>

### Customer Article Interaction based on Transaction Counts

In [25]:
# Create customer and article interaction dataframe based on total number of transactions made
customer_to_article_tdate = get_customer_article_interaction(customer_article_amt_df = transactions_data[['customer_id','article_id','t_dat']],
                                                            agg_col_name='t_dat')
print(customer_to_article_tdate.shape)     

(1051730, 3)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [26]:
customer_to_article_tdate.head()

,customer_id,article_id,total_no_of_transactions
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043,0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007,0
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0448509014,0
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0719530003,0
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0516859008,0


In [27]:
# Generate customer_article_interaction_matrix for train data
customer_to_article_interaction_tdat = get_interaction_matrix(customer_to_article_tdate, "customer_id", "article_id", "total_no_of_transactions", \
                                                            customer_to_index_mapping, article_to_index_mapping)

In [28]:
customer_to_article_interaction_tdat

<1371980x105542 sparse matrix of type '<class 'numpy.int8'>'
	with 1051730 stored elements in COOrdinate format>

## Hyperparameter Tuning using Random Search

In [29]:
def sample_hyperparameters():
    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 50),
        }

### Sampling Hyperparmeters Function

### Perform Random Search

Train and Test Interactions are provided as input parameters to the function including the random samples to generate and number of threads to use to perform model training.  

Output would be the precision score, set of hyperprameters and the model

In [30]:
def random_search(train_interactions, test_interactions, num_samples=10, num_threads=1):
    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams)
        model.fit(train_interactions, epochs=num_epochs, num_threads=num_threads)

        score = precision_at_k(model, test_interactions, train_interactions=train_interactions, k=12, num_threads=num_threads).mean()
        
        print(score)

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

### Initiating Storage Dictionary

In [31]:
optimized_dict={}

### Splitting the primary dataset into train and test sets based on amount spent

In [32]:
sparse_customer_article_train, sparse_customer_article_test = random_train_test_split(customer_to_article_interaction_amt, test_percentage=0.2, random_state=42)

In [33]:
sparse_customer_article_train

<1371980x105542 sparse matrix of type '<class 'numpy.int16'>'
	with 841384 stored elements in COOrdinate format>

In [34]:
sparse_customer_article_test

<1371980x105542 sparse matrix of type '<class 'numpy.int16'>'
	with 210346 stored elements in COOrdinate format>

In [35]:
(score, hyperparams, model) = max(random_search(train_interactions = sparse_customer_article_train, 
                                                test_interactions = sparse_customer_article_test, 
                                                num_threads = 4), key=lambda x: x[0])

0.0038191092
0.008680925
0.01146975
0.009560195
0.0043551703
0.004215272
0.004335559
0.006164051
0.00057332404
0.008816901


In [36]:
print("Best score {} at {}".format(score, hyperparams))

Best score 0.011469749733805656 at {'no_components': 60, 'learning_schedule': 'adagrad', 'loss': 'bpr', 'learning_rate': 0.024169871051368697, 'item_alpha': 1.5038748403807634e-08, 'user_alpha': 5.035922243112898e-09, 'max_sampled': 11, 'num_epochs': 35}


In [37]:
optimized_dict['Amount_Spent'] = {'score': score, 
                                  'params': hyperparams}

### Splitting the primary dataset into train and test sets based on transaction count

In [38]:
sparse_customer_article_train, sparse_customer_article_test = random_train_test_split(customer_to_article_interaction_tdat, test_percentage=0.2, random_state=42)

In [39]:
sparse_customer_article_train

<1371980x105542 sparse matrix of type '<class 'numpy.int8'>'
	with 841384 stored elements in COOrdinate format>

In [40]:
sparse_customer_article_test

<1371980x105542 sparse matrix of type '<class 'numpy.int8'>'
	with 210346 stored elements in COOrdinate format>

In [41]:
(score, hyperparams, model) = max(random_search(train_interactions = sparse_customer_article_train, 
                                                test_interactions = sparse_customer_article_test, 
                                                num_threads = 4), key=lambda x: x[0])

0.0007256439
0.002879041
0.0012950456
0.00088188605
0.0054632486
0.0027450258
0.005471094
0.0013231561
0.003609915
0.0030163252


In [42]:
print("Best score {} at {}".format(score, hyperparams))

Best score 0.00547109404578805 at {'no_components': 31, 'learning_schedule': 'adagrad', 'loss': 'warp-kos', 'learning_rate': 0.06666288591300393, 'item_alpha': 1.22364339580641e-08, 'user_alpha': 2.336263719204391e-09, 'max_sampled': 14, 'num_epochs': 42}


In [43]:
optimized_dict['Transaction_Counts'] = {'score': score, 
                                       'params': hyperparams}

In [44]:
print(optimized_dict)

{'Amount_Spent': {'score': 0.01146975, 'params': {'no_components': 60, 'learning_schedule': 'adagrad', 'loss': 'bpr', 'learning_rate': 0.024169871051368697, 'item_alpha': 1.5038748403807634e-08, 'user_alpha': 5.035922243112898e-09, 'max_sampled': 11, 'num_epochs': 35}}, 'Transaction_Counts': {'score': 0.005471094, 'params': {'no_components': 31, 'learning_schedule': 'adagrad', 'loss': 'warp-kos', 'learning_rate': 0.06666288591300393, 'item_alpha': 1.22364339580641e-08, 'user_alpha': 2.336263719204391e-09, 'max_sampled': 14, 'num_epochs': 42}}}


## Saving the Optimized Params

In [45]:
with open('optimized_dict.pkl', 'wb') as f:
    pickle.dump(optimized_dict, f)